# Введение в обработку естественного языка
## Курсовой проект

**Установим необходимые библиотеки**

In [1]:
!pip install python-telegram-bot --upgrade

     |████████████████████████████████| 430kB 7.2MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
     |████████████████████████████████| 2.6MB 39.6MB/s 


In [2]:
!pip install pymorphy2

     |████████████████████████████████| 61kB 3.9MB/s 
     |████████████████████████████████| 8.2MB 8.3MB/s 


In [3]:
!pip install stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp36-none-any.whl size=32916 sha256=b9dd669b08b143769d849686fdd2f30e294db31783368832c38e55013b1afcb5
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words


In [4]:
!pip install annoy

     |████████████████████████████████| 655kB 5.1MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp36-cp36m-linux_x86_64.whl size=390358 sha256=19154639fcff88438e26022c2253971ce55b7c7cfec74a85e621c79af7dc9d5c
  Stored in directory: /root/.cache/pip/wheels/3a/c5/59/cce7e67b52c8e987389e53f917b6bb2a9d904a03246fadcb1e
Successfully built annoy


**Подключим необходимые библиотеки к проекту**

In [5]:
import numpy as np
import pandas as pd

import os
import annoy
import pickle
import string

from tqdm.notebook import tqdm_notebook
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec, FastText

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters

**Подключим Гугл драйв**

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Укажем актуальные пути к файлам**

In [7]:
products_dataset = pd.read_csv("/content/drive/My Drive/Colab Notebooks/ProductsDataset.csv")
input_file = "/content/drive/My Drive/Colab Notebooks/Otvety.txt"
output_file = "/content/drive/My Drive/Colab Notebooks/prepared_answers.txt"

**Создадим полезные функции**

In [8]:
def preprocess_txt(line):
    """Строковый препроцессинг текстовой строки"""
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [9]:
def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in txt:
        if word in modelFT:
            vector_ft += modelFT[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    return vector_ft / n_ft

**Основная часть**

In [10]:
assert True

#Small preprocess of the answers

question = None
written = False

with open(output_file, "w", encoding="utf8") as fout:
    with open(input_file, "r", encoding="utf8") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

In [11]:
%%time

assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open(input_file, "r", encoding="utf8") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

CPU times: user 37min 28s, sys: 13.9 s, total: 37min 42s
Wall time: 37min 46s


In [12]:
sentences = [i for i in sentences if len(i) > 2]
modelFT = FastText(sentences=sentences, size=100, min_count=1, window=5)
modelFT.save("ft_model")

In [13]:
%%time

modelFT = FastText.load("ft_model")
ft_index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

with open(output_file, "r", encoding="utf8") as f:
    for line in tqdm_notebook(f):
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelFT:
                vector_ft += modelFT[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)
            
        counter += 1

ft_index.build(10)
ft_index.save('speaker.ann')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



CPU times: user 1h 32min 56s, sys: 53.6 s, total: 1h 33min 50s
Wall time: 1h 32min 50s


In [14]:
ft_index = annoy.AnnoyIndex(100, 'angular')
ft_index.load('speaker.ann')

True

In [15]:
ft_index.get_nns_by_vector(np.zeros(100), 2)

[2111, 2866]

In [16]:
products_dataset['text'] = products_dataset['title'] + " " + products_dataset["descrirption"]
products_dataset['text'] = products_dataset['text'].apply(lambda x: preprocess_txt(str(x)))
products_dataset.head()

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[юбка, детский, orby, новый, носить, реал, кра..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[ботильон, новыепривезти, чехия, указать, разм..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[брюки, размер, 4042, брюки, новый, знать, мер..."
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[продать, детский, шапка, продать, шапкикажда,..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[блузка, темносиний, 42, размерсостояние, отли..."


In [17]:
vectorizer = CountVectorizer(ngram_range=(1, 2))

In [18]:
idxs = set(np.random.randint(0, len(index_map), len(products_dataset)))
negative_texts = [" ".join(preprocess_txt(index_map[i])) for i in idxs]
positive_texts = [" ".join(val) for val in products_dataset['text'].values]

In [19]:
dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

In [20]:
X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, 
                                                    stratify=labels, random_state=13)

In [21]:
x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

lr = LogisticRegression().fit(x_train_vec, y_train)

In [22]:
accuracy_score(y_true=y_test, y_pred=lr.predict(x_test_vec))

0.9812951679183789

In [23]:
assert True

with open("/content/drive/My Drive/Colab Notebooks/index_speaker.pkl", "wb") as f:
    pickle.dump(index_map, f)

In [24]:
tfidf_vect = TfidfVectorizer().fit(X_train)

In [25]:
np.mean(tfidf_vect.idf_)

10.920510524741648

In [26]:
idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}
idfs

{'девочка': 9.543339292634592,
 'нравиться': 6.957627943724626,
 'самый': 9.995324416377649,
 'написать': 11.248087384873017,
 'десятка': 10.842622276764853,
 'письмо': 11.248087384873017,
 'служба': 11.248087384873017,
 'поддержка': 11.248087384873017,
 'удаться': 11.248087384873017,
 'выпытать': 10.842622276764853,
 'ответ': 11.248087384873017,
 'поменять': 10.331796652998863,
 'пароль': 11.248087384873017,
 'выслать': 11.248087384873017,
 'почтовый': 11.248087384873017,
 'ящик': 11.248087384873017,
 'порекомендовать': 11.248087384873017,
 'сменить': 10.842622276764853,
 'проц': 11.248087384873017,
 'двухъядерный': 11.248087384873017,
 'торрент': 11.248087384873017,
 'искать': 11.248087384873017,
 'сыворотка': 11.248087384873017,
 'лицо': 10.842622276764853,
 'гиалуроновый': 11.248087384873017,
 'кислота': 10.842622276764853,
 'обеспечивать': 11.248087384873017,
 'увлажнение': 10.554940204313072,
 'кожа': 11.248087384873017,
 'течение': 11.248087384873017,
 'час': 11.248087384873017,

In [27]:
ft_index_shop = annoy.AnnoyIndex(100 ,'angular')

midf = np.mean(tfidf_vect.idf_)

index_map_shop = {}
counter = 0

for i in tqdm_notebook(range(len(products_dataset))):
    n_ft = 0
    index_map_shop[counter] = (products_dataset.loc[i, "title"], products_dataset.loc[i, "image_links"])
    vector_ft = np.zeros(100)
    for word in products_dataset.loc[i, "text"]:
        if word in modelFT:
            vector_ft += modelFT[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_shop.add_item(counter, vector_ft)
    counter += 1

ft_index_shop.build(10)
ft_index_shop.save('shop.ann')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


True

In [28]:
assert True

with open("/content/drive/My Drive/Colab Notebooks/index_shop.pkl", "wb") as f:
    pickle.dump(index_map, f)

In [29]:
ft_index_shop.get_nns_by_vector(np.zeros(100), 1, include_distances=True)

([17], [1.4142135381698608])

**Запуск бота**

In [30]:
start_msg = 'Привет. Я бот. Меня зовут Crazy_Bot. Также, широко известен в узких ' \
'кругах как my_crazy_777_bot. Чем могу помочь?'

In [31]:
updater = Updater(token='1457609008:AAFyGWKlUmmRdjyQe2_41PqW-15ltCwea9Q') # Токен API к Telegram
dispatcher = updater.dispatcher

In [32]:
def startCommand(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text=start_msg)

In [33]:
def textMessage(bot, update):
    
    input_txt = preprocess_txt(update.message.text)
    vect = vectorizer.transform([" ".join(input_txt)])
    prediction = lr.predict(vect)
    
    if prediction[0] == 1:
        vect_ft = embed_txt(input_txt, idfs, midf)
        ft_index_shop_val = ft_index_shop.get_nns_by_vector(vect_ft, 5)
        for item in ft_index_shop_val:
            title, image = index_map_shop[item]
            bot.send_message(chat_id=update.message.chat_id, text="title: {} image: {}".format(title, image))
        return
    vect_ft = embed_txt(input_txt, {}, 1)
    ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
    if distances[0] > 0.2:
        print(distances[0])
        bot.send_message(chat_id=update.message.chat_id, text="Моя твоя не понимать")
        return
    bot.send_message(chat_id=update.message.chat_id, text=index_map[ft_index_val[0]])


In [34]:
# Хендлеры
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
# Добавляем хендлеры в диспатчер
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
# Начинаем поиск обновлений
updater.start_polling(clean=True)
# Останавливаем бота на Ctrl + C
updater.idle()

No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/telegram/ext/dispatcher.py", line 432, in process_update
    handler.handle_update(update, self, check, context)
  File "/usr/local/lib/python3.6/dist-packages/telegram/ext/handler.py", line 156, in handle_update
    return self.callback(update, context)
  File "<ipython-input-33-de1fac5b6c3e>", line 3, in textMessage
    input_txt = preprocess_txt(update.message.text)
AttributeError: 'CallbackContext' object has no attribute 'message'
No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/telegram/ext/dispatcher.py", line 432, in process_update
    handler.handle_update(update, self, check, context)
  File "/usr/local/lib/python3.6/dist-packages/telegram/ext/handler.py", line 156, in handle_update
    return self.callback(update, context)
  File "<ipython-input-33-de1fac5b6